In [ ]:
import os
from zipfile import ZipFile
import duckdb

In [42]:
## 31 GB of CSV fiels 2020-01 to 2025-03
# 
years = ['2020', '2021', '2022', '2023', '2024', '2025']
root = "../data/bike.csv"
dbpath = "../data/bike.db" 

# create a duckdb database
con = duckdb.connect(dbpath)


In [12]:
duckdb.sql("""
SELECT COUNT(*) FROM read_csv_auto('../data/bike.csv/2020/*.csv')
""").show()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│     19562314 │
└──────────────┘



In [13]:
duckdb.sql("""
SELECT COUNT(*) FROM read_csv_auto('../data/bike.csv/*/*.csv')
""").show()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│    163361883 │
└──────────────┘



In [ ]:

for year in years:
    con.execute("""
        CREATE OR REPLACE TABLE  stg_trips (
            ride_id VARCHAR,
            rideable_type VARCHAR,
            started_at TIMESTAMP,
            ended_at TIMESTAMP,
            start_station_name VARCHAR,
            start_station_id VARCHAR,
            end_station_name VARCHAR,
            end_station_id VARCHAR,
            start_lat DOUBLE,
            start_lng DOUBLE,
            end_lat DOUBLE,
            end_lng DOUBLE,
            member_casual VARCHAR
        );
        """)
    con.execute(f"""
        COPY stg_trips FROM '../data/bike.csv/{year}/*.csv'
        (DELIMITER ',', HEADER, NULL '', ENCODING 'utf8');
        """)
    con.execute(f"""
        COPY stg_trips TO '../data/bike.parquet/{year}.parquet'
    """)




FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

...

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [35]:
con.execute("""
SELECT filename, COUNT(*) FROM read_parquet('../data/bike.parquet/*', filename=True)
GROUP BY filename
""").fetchall()

[('../data/bike.parquet/2024.parquet', 35073137),
 ('../data/bike.parquet/2023.parquet', 35107030),
 ('../data/bike.parquet/2025.parquet', 7324003),
 ('../data/bike.parquet/2020.parquet', 19562314),
 ('../data/bike.parquet/2021.parquet', 27130122),
 ('../data/bike.parquet/2022.parquet', 29838806)]

In [28]:
con.execute("""
        CREATE OR REPLACE TABLE stg_trips_V1 (
            ride_id VARCHAR,
            rideable_type VARCHAR,
            started_at TIMESTAMP,
            ended_at TIMESTAMP,
            start_station_name VARCHAR,
            start_station_id VARCHAR,
            end_station_name VARCHAR,
            end_station_id VARCHAR,
            start_lat DOUBLE,
            start_lng DOUBLE,
            end_lat DOUBLE,
            end_lng DOUBLE,
            member_casual VARCHAR
        );
        """)

con.execute("""
        CREATE OR REPLACE TABLE stg_trips_V2 (
            "0" VARCHAR,
            ride_id VARCHAR,
            rideable_type VARCHAR,
            started_at TIMESTAMP,
            ended_at TIMESTAMP,
            start_station_name VARCHAR,
            start_station_id VARCHAR,
            end_station_name VARCHAR,
            end_station_id VARCHAR,
            start_lat DOUBLE,
            start_lng DOUBLE,
            end_lat DOUBLE,
            end_lng DOUBLE,
            member_casual VARCHAR
        );
        """)

con.execute(f"""
        COPY stg_trips_V1 FROM '../data/bike.csv/2024/*.csv'
        (DELIMITER ',', HEADER, NULL '', ENCODING 'utf8');
        """)

con.execute(f"""
        COPY stg_trips_V2 FROM '../data/bike.csv/2024v2/*.csv'
        (DELIMITER ',', HEADER, NULL '', ENCODING 'utf8');
        """)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [32]:
con.execute("""
SELECT COUNT(*) FROM stg_trips_V1
UNION ALL
SELECT COUNT(*) FROM stg_trips_V2
""").fetchall()

[(35073137,), (9326471,)]

In [36]:
con.execute("""
COPY (
SELECT * FROM stg_trips_V1
UNION ALL
SELECT * EXCLUDE("0") FROM stg_trips_V2
)TO '../data/bike.parquet/2024.parquet'
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [37]:
con.execute("""
SELECT filename, COUNT(*) FROM read_parquet('../data/bike.parquet/*', filename=True)
GROUP BY filename
""").fetchall()

[('../data/bike.parquet/2024.parquet', 44399608),
 ('../data/bike.parquet/2022.parquet', 29838806),
 ('../data/bike.parquet/2020.parquet', 19562314),
 ('../data/bike.parquet/2021.parquet', 27130122),
 ('../data/bike.parquet/2023.parquet', 35107030),
 ('../data/bike.parquet/2025.parquet', 7324003)]

In [38]:
#Let's build a view based on the parquet files
con.execute("""
CREATE OR REPLACE VIEW trips_view  AS   
SELECT * FROM read_parquet('../data/bike.parquet/*', filename=True)
""")
con.execute("""
SELECT filename, COUNT(*) FROM trips_view
GROUP BY filename
""").fetchall()

[('../data/bike.parquet/2024.parquet', 44399608),
 ('../data/bike.parquet/2020.parquet', 19562314),
 ('../data/bike.parquet/2021.parquet', 27130122),
 ('../data/bike.parquet/2023.parquet', 35107030),
 ('../data/bike.parquet/2025.parquet', 7324003),
 ('../data/bike.parquet/2022.parquet', 29838806)]

In [39]:
con.execute(""" SHOW TABLES; """).fetchall()

[('stg_trips',), ('stg_trips_V1',), ('stg_trips_V2',), ('trips_view',)]

In [43]:
con.execute("DESCRIBE trips_view").fetchall()

[('ride_id', 'VARCHAR', 'YES', None, None, None),
 ('rideable_type', 'VARCHAR', 'YES', None, None, None),
 ('started_at', 'TIMESTAMP', 'YES', None, None, None),
 ('ended_at', 'TIMESTAMP', 'YES', None, None, None),
 ('start_station_name', 'VARCHAR', 'YES', None, None, None),
 ('start_station_id', 'VARCHAR', 'YES', None, None, None),
 ('end_station_name', 'VARCHAR', 'YES', None, None, None),
 ('end_station_id', 'VARCHAR', 'YES', None, None, None),
 ('start_lat', 'DOUBLE', 'YES', None, None, None),
 ('start_lng', 'DOUBLE', 'YES', None, None, None),
 ('end_lat', 'DOUBLE', 'YES', None, None, None),
 ('end_lng', 'DOUBLE', 'YES', None, None, None),
 ('member_casual', 'VARCHAR', 'YES', None, None, None),
 ('filename', 'VARCHAR', 'YES', None, None, None)]

In [46]:
%%timeit
con.execute(f""" SELECT COUNT(*) FROM trips_view WHERE start_station_id = '5788.12' """).fetchall()

260 ms ± 2.33 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [51]:
con.execute(f""" SELECT COUNT(*) FROM read_csv_auto('../data/bike.csv/2021/*.csv') WHERE start_station_id = '5788.12' """).fetchall()

ConversionException: Conversion Error: CSV Error on Line: 108117
Original Line: CC6690970EF5B9AD,classic_bike,2021-01-28 15:02:30.592,2021-01-28 15:12:21.970,Pier 40 Dock Station,SYS035,Carmine St & 6 Ave,5763.03,40.72866,-74.01198,40.730386,-74.00215,member
Error when converting column "start_station_id". Could not convert string "SYS035" to 'DOUBLE'

Column start_station_id is being converted as type DOUBLE
This type was auto-detected from the CSV file.
Possible solutions:
* Override the type for this column manually by setting the type explicitly, e.g. types={'start_station_id': 'VARCHAR'}
* Set the sample size to a larger value to enable the auto-detection to scan more values, e.g. sample_size=-1
* Use a COPY statement to automatically derive types from an existing table.

  file = ../data/bike.csv/2021/202101-citibike-tripdata_1.csv
  delimiter = , (Auto-Detected)
  quote = " (Auto-Detected)
  escape = " (Auto-Detected)
  new_line = \n (Auto-Detected)
  header = true (Auto-Detecte

In [52]:
con.close()